In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
mp4_path = r"/content/drive/MyDrive/Video_summarisation/SumMe/videos/LH44_7.mp4"

: 

In [ ]:
import os
_,file_name = os.path.split(mp4_path)
file_name = file_name[:-4]

: 

###Taking user summary

In [ ]:
import pandas as pd
import numpy as np

: 

In [ ]:
summary_dir = r'/content/drive/MyDrive/Video_summarisation/SumMe/user_frames'
summary_file = summary_dir+'/'+file_name+'.csv'

: 

In [ ]:
df_user_summary = pd.read_csv(summary_file)

: 

In [ ]:
user_summary = np.clip(df_user_summary.to_numpy(), a_min = 0, a_max = 1)

: 

In [ ]:
sum(user_summary[:,0])

: 

# unsupervised leaning

1. Segment each video into shots
2. Extract features from each shot, including low-level features (such as color and motion) and high-level semantic features (such as object detection and scene recognition)
3. Cluster the shots based on their feature representations to group similar shots together
4. Identify key frames within each cluster by selecting frames that best represent the cluster
5. Order the key frames based on their importance score, which is calculated using a combination of visual and semantic information
6. Generate a summary by selecting the top-ranked key frames

## Step1

In [ ]:
import cv2
def extract_shots(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    # # Group frames into shots with fixed interval of 30 frames
    # shots = []
    # start_frame = 0
    # print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # while True:
    #     end_frame = start_frame + 29
    #     if end_frame >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
    #         end_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1
    #     shots.append((int(start_frame), int(end_frame)))
    #     if end_frame >= cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1:
    #         break
    #     start_frame = end_frame + 1

    # cap.release()

    return total_frames


: 

## step 2

there are 5 features that are extracted from each frame of a shot. The first three features are color features, representing the average color of the frame in the RGB color space. The last two features are motion features, representing the magnitude and direction of the optical flow between the current and previous frame in the shot. So, you are correct that there are 3 color features and 2 motion features.

In [ ]:
import cv2
import numpy as np

def compute_optical_flow(prev_gray, curr_gray, prev_pts):
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

    # Filter out points for which the flow is not found
    good_pts = prev_pts[status==1]
    new_pts = curr_pts[status==1]

    # Find the average flow vector
    flow_vec = np.mean(new_pts - good_pts, axis=0)

    return flow_vec

: 

In [ ]:
import cv2
import numpy as np

def extract_features(video_path, shots):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    features = []

    for shot in shots:
        start_frame, end_frame = shot
        frame_count = end_frame - start_frame + 1

        # Skip shots with less than two frames
        if frame_count < 2:
            continue

        # Initialize feature vectors
        color_features = np.zeros((frame_count, 3), dtype=np.float32)
        motion_features = np.zeros((frame_count-1, 2), dtype=np.float32)

        # Read first frame of shot
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        ret, prev_frame = cap.read()
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

        # Compute color feature for first frame
        color_features[0] = cv2.mean(prev_frame)[:3]

        # Initialize feature points for Lucas-Kanade method
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

        # Iterate over frames in shot
        for i in range(1, frame_count):
            # Read current frame
            ret, curr_frame = cap.read()
            if not ret:
                break
            curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

            # Compute color feature
            color_features[i] = cv2.mean(curr_frame)[:3]

            # Compute motion feature using Lucas-Kanade method
            flow_vec = compute_optical_flow(prev_gray, curr_gray, prev_pts)
            motion_features[i-1, 0] = np.linalg.norm(flow_vec)
            motion_features[i-1, 1] = np.arctan2(flow_vec[1], flow_vec[0])

            prev_gray = curr_gray
            prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

        # Normalize motion features
        motion_features /= np.max(motion_features, axis=0)

        # Combine color and motion features
        shot_features = np.hstack((color_features[:-1]/255, motion_features))

        # shot_features = np.mean(shot_features,axis = 0)

        # Add shot features to list of features
        features.append(shot_features)

    cap.release()

    return features


: 

In [ ]:
import cv2
import numpy as np

def extract_features(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    features = []

    # Initialize feature vectors
    color_features = np.zeros((frame_count, 3), dtype=np.float32)
    motion_features = np.zeros((frame_count-1, 2), dtype=np.float32)

    # Read first frame
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Compute color feature for first frame
    color_features[0] = cv2.mean(prev_frame)[:3]

    # Initialize feature points for Lucas-Kanade method
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

    # Loop over frames
    for i in range(1, frame_count):
        # Read current frame
        ret, curr_frame = cap.read()
        if not ret:
            break
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Compute color feature
        color_features[i] = cv2.mean(curr_frame)[:3]

        # Compute motion feature using Lucas-Kanade method
        flow_vec = compute_optical_flow(prev_gray, curr_gray, prev_pts)
        motion_features[i-1, 0] = np.linalg.norm(flow_vec)
        motion_features[i-1, 1] = np.arctan2(flow_vec[1], flow_vec[0])

        prev_gray = curr_gray
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

    # Normalize motion features
    motion_features /= np.max(motion_features, axis=0)

    # Combine color and motion features
    features = np.hstack((color_features[:-1]/255, motion_features))

    cap.release()

    return features
import cv2
import numpy as np

def extract_features(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    features = []

    # Initialize feature vectors
    color_features = np.zeros((frame_count, 3), dtype=np.float32)
    motion_features = np.zeros((frame_count-1, 2), dtype=np.float32)

    # Read first frame
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Compute color feature for first frame
    color_features[0] = cv2.mean(prev_frame)[:3]

    # Initialize feature points for Lucas-Kanade method
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

    # Loop over frames
    for i in range(1, frame_count):
        # Read current frame
        ret, curr_frame = cap.read()
        if not ret:
            break
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Compute color feature
        color_features[i] = cv2.mean(curr_frame)[:3]

        # Compute motion feature using Lucas-Kanade method
        flow_vec = compute_optical_flow(prev_gray, curr_gray, prev_pts)
        motion_features[i-1, 0] = np.linalg.norm(flow_vec)
        motion_features[i-1, 1] = np.arctan2(flow_vec[1], flow_vec[0])

        prev_gray = curr_gray
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.01, minDistance=10, blockSize=3)

    # Normalize motion features
    motion_features /= np.max(motion_features, axis=0)

    # Combine color and motion features
    features = np.hstack((color_features[:-1]/255, motion_features))

    cap.release()

    return features


: 

## step 3

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def cluster_shots(features, num_clusters):
    stacked_features = features
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(stacked_features)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    cluster_sizes = np.bincount(labels)
    cluster_indices = np.split(np.argsort(labels), np.cumsum(cluster_sizes)[:-1])

    clusters = [list(ci) for ci in cluster_indices]
    
    return clusters, centroids, kmeans


: 

### other methods (dont use)

In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA

# def cluster_shots(features, num_clusters):
#     # Reduce the dimensionality of the feature vectors using PCA
#     stacked_features = np.vstack(features)
#     pca = PCA(n_components=2)
#     features_pca = pca.fit_transform(stacked_features)

#     # Perform clustering on the reduced feature vectors
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(features_pca)
#     labels = kmeans.labels_
#     centroids = kmeans.cluster_centers_

#     # Create list of clusters based on labels
#     clusters = [[] for _ in range(num_clusters)]
#     [clusters[label].append(i) for i, label in enumerate(labels)]
    
#     return clusters, centroids


: 

In [ ]:
# from sklearn.cluster import AgglomerativeClustering
# import numpy as np

# def cluster_shots(features, num_clusters):
#     stacked_features = np.vstack(features)
#     agg = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
#     labels = agg.fit_predict(stacked_features)
    
#     clusters = [[] for _ in range(num_clusters)]
#     cluster_sizes = np.bincount(labels, minlength=num_clusters)  # initialize cluster_sizes to have length num_clusters
#     cluster_sizes_cumsum = np.cumsum(cluster_sizes)
#     shot_indices = np.arange(len(features))
    
#     label_indices = np.argsort(labels)
#     current_cluster = 0
#     for i in range(len(label_indices)):
#         while i >= cluster_sizes_cumsum[current_cluster]:
#             current_cluster += 1
#         if current_cluster >= num_clusters:  # check if current_cluster is out of bounds
#             break
#         shot_index = shot_indices[label_indices[i]]
#         clusters[current_cluster].append(shot_index)
    
#     centroids = []
#     for i in range(num_clusters):
#         centroid = np.mean([features[j] for j in clusters[i]], axis=0)
#         centroids.append(centroid)

#     return clusters, centroids


: 

In [ ]:
# from sklearn.cluster import AgglomerativeClustering
# import numpy as np

# def cluster_shots(features, num_clusters):
#     # Reduce the dimensionality of the feature vectors using PCA
#     stacked_features = np.vstack(features)
#     pca = PCA(n_components=5)
#     features_pca = pca.fit_transform(stacked_features)
#     agg = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
#     labels = agg.fit_predict(features_pca)
    
#     clusters = [[] for _ in range(num_clusters)]
#     cluster_sizes = np.bincount(labels)
#     cluster_sizes_cumsum = np.cumsum(cluster_sizes)
#     shot_indices = np.arange(len(features))
    
#     label_indices = np.argsort(labels)
#     current_cluster = 0
#     for i in range(len(label_indices)):
#         while i >= cluster_sizes_cumsum[current_cluster]:
#             current_cluster += 1
#         shot_index = shot_indices[label_indices[i]]
#         clusters[current_cluster].append(shot_index)

#     return clusters

: 

## step 4

### for full length video

In [ ]:
# def identify_keyframes(clusters, features):
#     keyframes = []
#     stacked_features = features

#     for cluster in clusters:
#         # Find representative shot for cluster (shot with smallest Euclidean distance to cluster centroid)
#         cluster_features = np.array([stacked_features[i] for i in cluster])
#         centroid = np.mean(cluster_features, axis=0)
#         distances = np.linalg.norm(cluster_features - centroid, axis=1)
#         representative_shot = cluster[np.argmin(distances)]

#         # Add keyframe for representative shot (first frame of representative shot)
#         keyframes.append(representative_shot)

#     return keyframes


: 

### for initial nsec of the video

In [ ]:
def pick_centroids(clusters, features):
    centroids = []

    stacked_features = features

    for i, cluster in enumerate(clusters):
        # Find cluster centroid
        cluster_features = np.array([stacked_features[i] for i in cluster])
        centroid = np.mean(cluster_features, axis=0)

        # Check if centroid meets criteria (average color and high motion magnitude)
        color_features = centroid[:3]
        motion_features = centroid[3:]
        motion_mag = np.linalg.norm(motion_features)
        c = 0.49
        m = 0.7
        if color_features[0] > c and color_features[1] > c and color_features[2] > c and motion_mag > m:
            centroids.append((i,centroid))

    centroids = np.array(centroids)
    sorted_centroids = sorted(centroids, key=lambda x: np.mean(x[1][:4]), reverse=True)
    
    return sorted_centroids


: 

In [ ]:
def pick_keyframes_from_centroids(centroids, clusters, features):
    keyframes = []
    stacked_features = features

    for i in range(len(centroids)):
        centroid_index = centroids[i][0]  # get the centroid index from the sorted centroids array
        cluster = clusters[centroid_index]
        centroid = centroids[i][1]

        # Find distances from each shot to centroid
        distances = np.linalg.norm(stacked_features[cluster] - centroid, axis=1)
        distances.sort()

        # Select closest shots to centroid
        closest_shots = [cluster[j] for j in np.argsort(distances)[:1]]

        keyframes.append(closest_shots)
    return keyframes

: 

In [ ]:
def create_keyshots(d,nframes,keyframes):
    keyframe_list = []
    for i in keyframes:
        l = i - d
        u = i + d
        r = np.arange(l,u)
        keyframe_list.append(r)
    keyshots = np.unique(np.clip(np.hstack(keyframe_list), a_min = 0, a_max = nframes-2))

    return keyshots

: 

## step 5

In [ ]:
def calculate_importance_score(keyframes, features):
    stacked_features = features
    # Compute visual scores (based on color and motion features)
    color_features = np.array([stacked_features[i][:3] for i in keyframes])
    motion_features = np.array([stacked_features[i][3:] for i in keyframes])
    color_scores = np.linalg.norm(color_features, axis=1)
    motion_scores = np.linalg.norm(motion_features, axis=1)
    visual_scores = color_scores + motion_scores

    # # Compute semantic scores (based on cluster labels and semantic scores)
    # cluster_sizes = [len(cluster) for cluster in labels]
    # cluster_weights = np.array(cluster_sizes) / np.sum(cluster_sizes)
    # semantic_scores = np.array([semantic_scores[i] for i in labels])
    # semantic_scores_weighted = np.sum(semantic_scores * cluster_weights, axis=1)
    # semantic_scores_normalized = (semantic_scores_weighted - np.min(semantic_scores_weighted)) / (np.max(semantic_scores_weighted) - np.min(semantic_scores_weighted))
    # semantic_scores_normalized = np.clip(semantic_scores_normalized, 0, 1)

    # Combine visual and semantic scores to get importance scores
    # importance_scores = 0.5 * visual_scores + 0.5 * semantic_scores_normalized

    importance_scores = np.round(visual_scores,3)

    # # Sort keyframes by importance score
    # sorted_indices = np.argsort(importance_scores)[::-1]
    # sorted_keyframes = [keyframes[i] for i in sorted_indices]

    return importance_scores

: 

## step 6

In [ ]:
def generate_summary(keyframes):
    # Sort keyframes by index (to ensure temporal order)
    keyframes = sorted(keyframes)

    # Select top-ranked keyframes based on their index
    summary = keyframes

    return summary

: 

## implementation

In [ ]:
import cv2
import numpy as np

 # Step 1: Segment the video into shots
video_file = mp4_path
nframes = extract_shots(video_file)
print('step_1_done')

: 

In [ ]:
# Step 2: Extract features from each shot (color and motion)
features = extract_features(video_file)
print('step_2_done')

: 

In [ ]:
# Step 3: Cluster shots based on their feature representations
num_clusters = int(.15*len(features))
# num_clusters = 75
clusters, centroids, kmeans = cluster_shots(features, num_clusters)
print('step_3_done')

: 

In [ ]:
# # Step 4: Identify keyframes within each cluster for full length
# keyframes = identify_keyframes(clusters, features)
# print('step_4_done')

: 

In [ ]:
# # Step 4: Identify keyframes within each cluster for first nsec
sorted_centroids = pick_centroids(clusters, features)
keyframes_list = pick_keyframes_from_centroids(sorted_centroids, clusters, features)
keyframes = np.concatenate(keyframes_list)

d = int(0.15*nframes/len(keyframes))
# d = 5
keyshots = create_keyshots(d,nframes,keyframes)
keyframes = keyshots.astype(int)
print('step_4_done')

: 

In [ ]:
# Step 5: Calculate importance scores
importance_scores = calculate_importance_score(keyframes, features)
print('step_5_done')

: 

In [ ]:
# Step 6: Generate summary by selecting top-ranked keyframes
summary = generate_summary(keyframes)
summary_unsup = summary
print('step_6_done')

: 

# plotting

In [ ]:
stacked_features = features

: 

In [ ]:
summary_array = np.zeros(len(stacked_features))
summary_array[summary] = 1
summary_array_unsup = summary_array

: 

In [ ]:
sum(summary_array)

: 

# Step 7 (comapring to user summary)

In [ ]:
np.arange(np.shape(user_summary)[1])

: 

In [ ]:
from sklearn.metrics import classification_report

jaccard_similarity = []
f1_scores = []
for i in np.arange(np.shape(user_summary)[1]):
    intersection = np.logical_and(user_summary[:,i].astype(bool), summary_array.astype(bool))
    union = np.logical_or(user_summary[:,i].astype(bool), summary_array.astype(bool))
    jaccard_similarity.append(np.sum(intersection) / np.sum(union)*100)
    f1_scores.append(classification_report(user_summary[:,1], summary_array))
print(jaccard_similarity)

: 

In [ ]:
import matplotlib.pyplot as plt
fig, [ax1,ax2] = plt.subplots(2,1)
ax1.plot(user_summary[:,10])
ax2.plot(summary_array)
ax1.title.set_text('user_summary')
ax2.title.set_text('generated_summary')
ax1.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_xlabel('Frame number')
fig.tight_layout()

: 

In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(np.shape(user_summary)[1]),jaccard_similarity)
ax.set_xticks(np.arange(0,np.shape(user_summary)[1]),np.arange(1,np.shape(user_summary)[1]+1))
ax.title.set_text('Similarity between user and generated summary')
ax.set_xlabel('User')
ax.set_ylabel('jaccard_similarity_%')

: 

# creating summary array for supervised learning

In [ ]:
s_stacked_features = stacked_features
for i in range(np.shape(user_summary)[1]-1):
    s_stacked_features = np.concatenate((s_stacked_features, stacked_features))

: 

In [ ]:
user_summary_array = user_summary.flatten(order='F')

: 

In [ ]:
max(user_summary_array)

: 

# supervised learning

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Load dataset
# X = np.load('features.npy')  # shape (n_frames, n_features)
# y = np.load('summary.npy')  # shape (n_frames,)
X = s_stacked_features
y = user_summary_array

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
# clf = LogisticRegression()
# clf = RandomForestClassifier(n_estimators=100, random_state=5)
# clf = SVC(kernel='rbf', random_state=42)
# clf = KNeighborsClassifier(n_neighbors=5)
clf = GaussianNB()

clf.fit(X_train, y_train)

# Evaluate the model on the testing set
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")


: 

### CNN and RNN (LSTM)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D

# Load dataset
# X = np.load('features.npy')  # shape (n_frames, n_features)
# y = np.load('summary.npy')  # shape (n_frames,)
X = s_stacked_features
y = user_summary_array

# Reshape input data for LSTM
n_timesteps, n_features = 1, X.shape[1]
X = X.reshape((X.shape[0], n_timesteps, n_features))

# Define LSTM model with multiple layers
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

# Train the model with additional regularization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min', min_lr=0.0001)

model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2)#, callbacks=[early_stop, reduce_lr])

: 

In [ ]:
model.summary()

: 

### DNN

In [ ]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Dense, Dropout

# # Load dataset
# # X = np.load('features.npy')  # shape (n_frames, n_features)
# # y = np.load('summary.npy')  # shape (n_frames,)
# X = s_stacked_features
# y = user_summary_array

# # Define DNN model with multiple layers
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model with additional regularization
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min', min_lr=0.0001)

# model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stop, reduce_lr])


: 

### RNN

In [ ]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout

# # Load dataset
# # X = np.load('features.npy')  # shape (n_frames, n_features)
# # y = np.load('summary.npy')  # shape (n_frames,)
# X = s_stacked_features
# y = user_summary_array

# # Reshape input data for LSTM
# n_timesteps, n_features = 1, X.shape[1]
# X = X.reshape((X.shape[0], n_timesteps, n_features))

# # Define LSTM model with multiple layers
# model = Sequential()
# model.add(LSTM(64, return_sequences=True, input_shape=(n_timesteps, n_features)))
# model.add(Dropout(0.2))
# model.add(LSTM(32, return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(16))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# # Train the model with additional regularization
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min', min_lr=0.0001)

# model.fit(X, y, epochs=20, batch_size=64, validation_split=0.3, callbacks=[early_stop, reduce_lr])


: 

### CM

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, classification_report

: 

In [ ]:
jaccard_similarity = []
for i in np.arange(np.shape(user_summary)[1]):
    intersection = np.logical_and(user_summary[:,i].astype(bool), summary_array.astype(bool))
    union = np.logical_or(user_summary[:,i].astype(bool), summary_array.astype(bool))
    jaccard_similarity.append(np.sum(intersection) / np.sum(union)*100)
print(jaccard_similarity)

: 

In [ ]:
# Load dataset
X = features
y = user_summary[:,0]

# Reshape input data for LSTM
n_timesteps, n_features = 1, X.shape[1]
X = X.reshape((X.shape[0], n_timesteps, n_features))

# Predict the summary array
y_pred = model.predict(X)
y_pred_binary = (y_pred > 0.23).astype(int) #0.225, 0.23


# Generate confusion matrix
cm = confusion_matrix(y, y_pred_binary)

# Generate classification report
target_names = ['0', '1']
print(classification_report(y, y_pred_binary, target_names=target_names))

: 

In [ ]:
# # Load dataset
# X = features


# # Reshape input data for LSTM
# n_timesteps, n_features = 1, X.shape[1]
# X = X.reshape((X.shape[0], n_timesteps, n_features))

# # Predict the summary array
# y_pred = model.predict(X)
# y_pred_binary = (y_pred > 0.23).astype(int) #0.225, 0.23

: 

In [ ]:
target_names = ['0', '1']
print(classification_report(y, y_pred_binary, target_names=target_names))

: 

In [ ]:
cm

: 

In [ ]:
sum(y_pred_binary)

: 

In [ ]:
indices,_ = np.where(y_pred_binary == 1)

: 

In [ ]:
len(features)

: 

In [ ]:
d = int(0.15*nframes/len(indices))
keyshots = create_keyshots(d,nframes,indices)
keyframes = keyshots.astype(int)
summary = keyframes
summary_sup = summary

: 

In [ ]:
keyshots = np.zeros(len(features))
keyshots[keyframes] = 1
summary_array = keyshots
summary_array_sup = keyshots

: 

In [ ]:
jaccard_similarity = []
for i in np.arange(np.shape(user_summary)[1]):
    intersection = np.logical_and(user_summary[:,i].astype(bool), keyshots.astype(bool))
    union = np.logical_or(user_summary[:,i].astype(bool), keyshots.astype(bool))
    jaccard_similarity.append(np.sum(intersection) / np.sum(union)*100)
print(jaccard_similarity)

: 

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(user_summary[:,10])
# plt.plot(keyshots)
fig, [ax1,ax2] = plt.subplots(2,1)
ax1.plot(user_summary[:,10])
ax2.plot(summary_array)
ax1.title.set_text('user_summary')
ax2.title.set_text('generated_summary')
ax1.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_xlabel('Frame number')
fig.tight_layout()

: 

In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(np.shape(user_summary)[1]),jaccard_similarity)
ax.set_xticks(np.arange(0,np.shape(user_summary)[1]),np.arange(1,np.shape(user_summary)[1]+1))
ax.title.set_text('Similarity between user and generated summary')
ax.set_xlabel('User')
ax.set_ylabel('jaccard_similarity_%')

: 

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(user_summary[:,10])
# plt.plot(keyshots)
fig, [ax1,ax2] = plt.subplots(2,1)
ax1.plot(summary_array_unsup)
ax2.plot(summary_array_sup)
ax1.title.set_text('unsupervised_learning')
ax2.title.set_text('supervised_learning')
ax1.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_yticklabels(['','Not selected','','','','selected'])
ax2.set_xlabel('Frame number')
fig.tight_layout()

: 

In [ ]:
df_ypred = pd.DataFrame(y_pred, columns=["y_pred"])
df_ypred.describe()

: 

In [ ]:
# import cv2
# import imageio

# # Load video using OpenCV
# cap = cv2.VideoCapture(video_file)

# # Extract frames corresponding to summary frames and save as a new GIF
# out_file = 'summary.gif'
# writer = imageio.get_writer(out_file, mode='I', duration=0.1)

# for frame_idx in summary:
#     cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
#     ret, frame = cap.read()
#     if ret:
#         writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# cap.release()
# writer.close()

# print('gif created')

: 

#exporting the gif

In [ ]:
import imageio
import os
from skimage.transform import resize

# Load video using imageio
reader = imageio.get_reader(video_file)

# Extract frames corresponding to summary frames and save as a new GIF
out_file = '/content/drive/MyDrive/Video_summarisation/summary_supervised_random.gif'
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer(out_file, fps=fps)

for frame_idx in summary_sup:
    frame = reader.get_data(frame_idx)
    frame_resized = resize(frame, (480, 854))
    writer.append_data(frame)

writer.close()

print('gif_created')

: 